In [ ]:
from waymo_open_dataset import dataset_pb2

import json
import os

import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from waymo_open_dataset import label_pb2
from waymo_open_dataset.protos import camera_segmentation_pb2 as cs_pb2
from waymo_open_dataset.utils import box_utils


import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path


WOPD_DATA_ROOT = Path("/media/nvme1/waymo_perception/training")


tfrecords_file_list = list(WOPD_DATA_ROOT.glob("*.tfrecord"))

In [ ]:
import io
from pyquaternion import Quaternion

from d123.common.geometry.base import StateSE3
from d123.common.geometry.bounding_box.bounding_box import BoundingBoxSE3

from waymo_open_dataset.utils import frame_utils


# Frame attributes:
#   context: <class 'waymo_open_dataset.dataset_pb2.Context'>
#   timestamp_micros: <class 'int'>
#   pose: <class 'waymo_open_dataset.dataset_pb2.Transform'>
#   images: List with 5 images
#   lasers: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 5
#   laser_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   projected_lidar_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   camera_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   no_label_zones: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_features: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_pose_offset: <class 'waymo_open_dataset.protos.vector_pb2.Vector3d'>

file_idx = 0
pathname = tfrecords_file_list[file_idx]
dataset = tf.data.TFRecordDataset(pathname, compression_type="")
num_frames = sum(1 for _ in dataset)


def read_jpg_image(data: bytes) -> np.ndarray:
    """Read a JPEG image from bytes and return it as a numpy array."""
    image = Image.open(io.BytesIO(data))
    return np.array(image)


ego_state_se3s = []
front_images = []
dataset = tf.data.TFRecordDataset(pathname, compression_type="")

boxes = []

for frame_idx, data in enumerate(dataset):

    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    # print(frame.camera_labels)
    print("Frame attributes:")
    for field in frame.DESCRIPTOR.fields:
        field_name = field.name
        if hasattr(frame, field_name):
            value = getattr(frame, field_name)
            if field_name != "images":  # Don't print the whole image data
                print(f"  {field_name}: {type(value)}")
                if hasattr(value, "__len__") and not isinstance(value, (str, bytes)):
                    print(f"    Length: {len(value)}")
            else:
                print(f"  {field_name}: List with {len(value)} images")


    # # 1. pose
    pose = np.array(frame.pose.transform).reshape(4, 4)
    yaw_pitch_roll = Quaternion(matrix=pose[:3, :3]).yaw_pitch_roll
    ego_state_se3s.append(
        np.array(
            [
                pose[0, 3],  # x
                pose[1, 3],  # y
                pose[2, 3],  # z
                yaw_pitch_roll[2],  # yaw
                yaw_pitch_roll[1],  # pitch
                yaw_pitch_roll[0],  # roll
            ],
            dtype=np.float64,
        )
    )

    # # plt.show()
    if frame_idx == 0:
        break

ego_state_se3s = np.array(ego_state_se3s, dtype=np.float64)

In [ ]:
for frame_idx, data in enumerate(dataset):
    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    if frame_idx == 2:
        break

print("Ego")
ego_transform = np.array(frame.pose.transform).reshape(4, 4)
print(ego_transform[:3, 3])

# 1 [ 1.5441613  -0.02302364  2.11557864]
# 2 [1.49672397 0.0954948  2.11616463]
# 3 [ 1.49442485 -0.09637497  2.11519385]
# 4 [1.43213651 0.11612398 2.11625087]
# 5 [ 1.42936162 -0.11545043  2.1150792 ]


# frame.map_pose_offset

In [ ]:
plt.figure(figsize=(64, 20))


def plot_range_image_helper(data, name, layout, vmin=0, vmax=1, cmap="gray"):
    """Plots range image.

    Args:
      data: range image data
      name: the image title
      layout: plt layout
      vmin: minimum value of the passed data
      vmax: maximum value of the passed data
      cmap: color map
    """
    plt.subplot(*layout)
    plt.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.title(name)
    plt.grid(False)
    plt.axis("off")


def get_range_image(laser_name, return_index):
    """Returns range image given a laser name and its return index."""
    return range_images[laser_name][return_index]


# def show_range_image(range_image, layout_index_start=1):
#     """Shows range image.

#     Args:
#       range_image: the range image data from a given lidar of type MatrixFloat.
#       layout_index_start: layout offset
#     """
#     range_image_tensor = tf.convert_to_tensor(range_image.data)
#     range_image_tensor = tf.reshape(range_image_tensor, range_image.shape.dims)
#     lidar_image_mask = tf.greater_equal(range_image_tensor, 0)
#     range_image_tensor = tf.where(lidar_image_mask, range_image_tensor, tf.ones_like(range_image_tensor) * 1e10)
#     range_image_range = range_image_tensor[..., 0]
#     range_image_intensity = range_image_tensor[..., 1]
#     range_image_elongation = range_image_tensor[..., 2]
#     plot_range_image_helper(range_image_range.numpy(), "range", [8, 1, layout_index_start], vmax=75, cmap="gray")
#     plot_range_image_helper(
#         range_image_intensity.numpy(), "intensity", [8, 1, layout_index_start + 1], vmax=1.5, cmap="gray"
#     )
#     plot_range_image_helper(
#         range_image_elongation.numpy(), "elongation", [8, 1, layout_index_start + 2], vmax=1.5, cmap="gray"
#     )


def show_range_image(range_image, layout_index_start=1):
    """Shows range image.

    Args:
      range_image: the range image data from a given lidar of type MatrixFloat.
      layout_index_start: layout offset
    """
    range_image_tensor = np.array([data for data in range_image.data]).reshape(range_image.shape.dims)
    lidar_image_mask = np.greater_equal(range_image_tensor, 0)
    range_image_tensor = np.where(lidar_image_mask, range_image_tensor, np.ones_like(range_image_tensor) * 1e10)
    range_image_range = range_image_tensor[..., 0]
    range_image_intensity = range_image_tensor[..., 1]
    range_image_elongation = range_image_tensor[..., 2]
    plot_range_image_helper(range_image_range, "range", [8, 1, layout_index_start], vmax=75, cmap="gray")
    plot_range_image_helper(range_image_intensity, "intensity", [8, 1, layout_index_start + 1], vmax=1.5, cmap="gray")
    plot_range_image_helper(range_image_elongation, "elongation", [8, 1, layout_index_start + 2], vmax=1.5, cmap="gray")


frame.lasers.sort(key=lambda laser: laser.name)
show_range_image(get_range_image(open_dataset.LaserName.TOP, 0), 1)
# show_range_image(get_range_image(open_dataset.LaserName.TOP, 1), 4)

In [ ]:
from d123.common.datatypes.time.time_point import TimePoint


for frame_idx, data in enumerate(dataset):
    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    if frame_idx == 4:
        break
    break


# for calibration in frame.context.camera_calibrations:

frame.timestamp_micros, frame.images[0].pose_timestamp
# frame.images[0]

frame_timestamp = TimePoint.from_us(frame.timestamp_micros)
image_timestamp = TimePoint.from_s(frame.images[0].pose_timestamp)
frame_timestamp.time_s, frame_timestamp.time_s